In [1]:
# from sst and rainfall ---> rainfall
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
from netCDF4 import Dataset
import matplotlib.pyplot as plt 
import tensorflow.compat.v1 as tf
import keras.backend as K
from keras.models import Sequential
from keras.layers import Conv3D, Conv2D
from keras.layers import ConvLSTM2D
from keras.layers import Attention
from keras.layers import Input
from keras.models import Model
from keras.layers import Dropout, Lambda
from keras import optimizers
from keras.layers import Conv2DTranspose, TimeDistributed
from keras import callbacks
import xarray as xr

# Define global variables
lead_time = 3
save_path = "/home/cccr/roxy/matin/MTech_project/model/Conv-LSTM/7in4out/"
model_path = save_path + "ConvLstm_sst13rf_rf_3bob.h5"
add_data = "/home/cccr/roxy/matin/MTech_project/data/"

def preprocess_data(sequence, n_steps, channels):
    """
    Function to preprocess the data and prepare it for model training.
    
    Parameters:
        sequence (xarray.Dataset): Data sequence to be preprocessed.
        n_steps (int): Number of time steps in the input sequence.
        channels (int): Number of channels in the input sequence.
    
    Returns:
        tuple: A tuple containing the preprocessed input sequence (X) and output sequence (y).
    """
    # Select data for Jun, Jul, Aug, Sept
    sequence = sequence.sel(time=slice("1998-01-01","2016-12-31"))
    sequence = sequence.where(sequence.time.dt.month.isin([6,7,8,9]), drop=True)
    max = sequence.max()
    min = sequence.min()
    
    # Max-min normalization
    sequence = (sequence-min)/(max - min)
    sequence = np.array(sequence)
    
    # Exponential Space Transform
    valid_pts = np.where(sequence==sequence)
    sequence[valid_pts] = np.exp(sequence[valid_pts])
    sequence[valid_pts] = np.power(sequence[valid_pts],7)
    invalid_pts = np.where(sequence != sequence)
    sequence[invalid_pts] = 0
    
    # Prepare input and output sequences
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix + lead_time > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + lead_time]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


def create_input(channels, n_steps):
    """
    Function to create the input for the model.
    
    Parameters:
        channels (list): List of channels to be used in the input.
        n_steps (int): Number of time steps in the input sequence.
        
    Returns:
        numpy.ndarray: The input for the model.
    """
    stack = []
    for i in range(len(channels)):
        channel_data = xr.open_dataarray(f"/home/cccr/roxy/matin/MTech_project/data/{channels[i]}")
        input, _ = preprocess_data(channel_data, n_steps=n_steps, channels=channels[i])
        input = np.expand_dims(input, axis=2)
        print(f"Adding channel {i} with shape: {input.shape}")
        stack.append(input)
    out = np.dstack(stack)
    return out


channels = ["FilteredrfBOB_13lag.nc","FilteredSSTBOB_13.nc"]
steps = 7

# Create input for the model
input = create_input(channels, steps)

# Preprocess target data
channel_data = xr.open_dataarray(add_data + "FilteredrfBOB_13lag.nc")
_, target = preprocess_data(channel_data, n_steps=steps, channels=channels[-1])
target = np.expand_dims(target, axis=1)

# Roll axes for input and target
x = np.rollaxis(input, 4, 2)
x = np.rollaxis(x, 4, 2)

y = np.rollaxis(target, 3, 1)
y = np.rollaxis(y, 3, 1)

# Assert that input and target shapes are consistent
assert input.shape[0] == target.shape[0]
assert input.shape[-1] == target.shape[-1]
assert input.shape[-2] == target.shape[-2]


# Print shape of input and target
print("INPUT SHAPE --> ", input.shape)
print("TARGET SHAPE --> ", target.shape)

# Delete original input and target variables to free up memory
del input
del target

# Perform additional processing or modeling steps as needed
seq = tf.keras.Sequential()

# Add layers to the model
seq.add(ConvLSTM2D(filters=4, kernel_size=(3,3), padding='same', input_shape=(7,25,39,2), return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=8, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=8, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=16, kernel_size=(3,3), padding='same', return_sequences=True, data_format='channels_last'))
seq.add(ConvLSTM2D(filters=16, kernel_size=(3,3), padding='same', return_sequences=False, data_format='channels_last'))
seq.add(Conv2D(filters=15, kernel_size=(3,3), activation='relu', padding='same', data_format='channels_last'))
seq.add(Conv2D(filters=1, kernel_size=(3,3), activation='relu', padding='same', data_format='channels_last'))
# Create the query, value, and key inputs
query_input = Input(shape=(25,39,2), name='query_input')
value_input = Input(shape=(25,39,2), name='value_input')
key_input = Input(shape=(25,39,2), name='key_input')

# Pass the inputs to the Attention layer
attention_output = Attention(name='attention')([query_input, value_input, key_input])
Adam = tf.keras.optimizers.Adam(learning_rate=10**-4)
seq.compile(loss='mean_squared_error', optimizer=Adam, metrics=['mae'])

print(seq.summary())

# Define the number of training epochs
n_epochs = 200
logdir = save_path
# Define the TensorBoard callback for monitoring training progress
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir, histogram_freq=10, batch_size=10, write_graph=True)

# Define the ReduceLROnPlateau callback for reducing the learning rate when the model plateaus
lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=20, min_lr=10**-20)

# Define the ModelCheckpoint callback for saving the best model based on validation mean absolute error
checkpoint_callback_mae = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_mae',
                                                             mode='min', save_best_only=True)

# Define the ModelCheckpoint callback for saving the best model based on validation loss
checkpoint_callback_loss = tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss',
                                                              mode='min', save_best_only=True)

# Define the EarlyStopping callback for stopping training when the model stops improving
early_stop_callback = tf.keras.callbacks.EarlyStopping(patience=30, restore_best_weights=True)

# Define the TerminateOnNaN callback for terminating the training process if there is NaN value in any of the output
terminate_callback = tf.keras.callbacks.TerminateOnNaN()

# Train the model using the defined callbacks
history = seq.fit(x, y, epochs=n_epochs, validation_split=0.1,
                  batch_size=50, callbacks=[early_stop_callback,
                  checkpoint_callback_mae, checkpoint_callback_loss,
                  tb_callback, lr_callback, terminate_callback])
#Save the trained model
seq.save(model_path)

#Save the training history

np.save(f'{save_path}ConvLstm_sst13rf_rf_3bob.npy', history.history)               
              

Adding channel 0 with shape: (2308, 7, 1, 25, 39)
Adding channel 1 with shape: (2308, 7, 1, 25, 39)
INPUT SHAPE -->  (2308, 7, 2, 25, 39)
TARGET SHAPE -->  (2308, 1, 25, 39)


2023-03-14 16:32:43.760103: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d (ConvLSTM2D)    (None, 7, 25, 39, 4)      880       
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 7, 25, 39, 8)      3488      
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 7, 25, 39, 8)      4640      
                                                                 
 conv_lstm2d_3 (ConvLSTM2D)  (None, 7, 25, 39, 16)     13888     
                                                                 
 conv_lstm2d_4 (ConvLSTM2D)  (None, 25, 39, 16)        18496     
                                                                 
 conv2d (Conv2D)             (None, 25, 39, 15)        2175      
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 39, 1)         1

Epoch 44/200
42/42 [==============================] - 247s 6s/step - loss: 1041.5597 - mae: 20.9580 - val_loss: 928.2460 - val_mae: 19.2108 - lr: 1.0000e-04
Epoch 45/200
42/42 [==============================] - 252s 6s/step - loss: 1041.3079 - mae: 20.9640 - val_loss: 928.3148 - val_mae: 19.2658 - lr: 1.0000e-04
Epoch 46/200
42/42 [==============================] - 242s 6s/step - loss: 1041.2407 - mae: 21.0382 - val_loss: 927.9292 - val_mae: 19.2307 - lr: 1.0000e-04
Epoch 47/200
42/42 [==============================] - 253s 6s/step - loss: 1041.0759 - mae: 20.8978 - val_loss: 927.6008 - val_mae: 19.2022 - lr: 1.0000e-04
Epoch 48/200
42/42 [==============================] - 257s 6s/step - loss: 1041.0616 - mae: 21.0234 - val_loss: 927.5061 - val_mae: 19.2259 - lr: 1.0000e-04
Epoch 49/200
42/42 [==============================] - 256s 6s/step - loss: 1040.6384 - mae: 20.9438 - val_loss: 927.0360 - val_mae: 19.1593 - lr: 1.0000e-04
Epoch 50/200
42/42 [==============================] - 259s

42/42 [==============================] - 257s 6s/step - loss: 1033.2280 - mae: 20.8579 - val_loss: 918.3046 - val_mae: 19.0884 - lr: 1.0000e-04
Epoch 97/200
42/42 [==============================] - 255s 6s/step - loss: 1033.2496 - mae: 20.8604 - val_loss: 917.9109 - val_mae: 19.0314 - lr: 1.0000e-04
Epoch 98/200
42/42 [==============================] - 251s 6s/step - loss: 1033.1027 - mae: 20.8290 - val_loss: 917.8386 - val_mae: 19.0575 - lr: 1.0000e-04
Epoch 99/200
42/42 [==============================] - 257s 6s/step - loss: 1032.8436 - mae: 20.8321 - val_loss: 917.6370 - val_mae: 19.0473 - lr: 1.0000e-04
Epoch 100/200
42/42 [==============================] - 251s 6s/step - loss: 1032.6193 - mae: 20.7942 - val_loss: 917.4983 - val_mae: 19.0529 - lr: 1.0000e-04
Epoch 101/200
42/42 [==============================] - 252s 6s/step - loss: 1032.4763 - mae: 20.8291 - val_loss: 917.6172 - val_mae: 19.1104 - lr: 1.0000e-04
Epoch 102/200
42/42 [==============================] - 260s 6s/step -

Epoch 148/200
42/42 [==============================] - 253s 6s/step - loss: 673.9853 - mae: 14.5191 - val_loss: 555.2117 - val_mae: 12.4493 - lr: 1.0000e-04
Epoch 149/200
42/42 [==============================] - 244s 6s/step - loss: 611.4375 - mae: 13.5646 - val_loss: 521.8705 - val_mae: 12.3667 - lr: 1.0000e-04
Epoch 150/200
42/42 [==============================] - 252s 6s/step - loss: 570.4106 - mae: 13.0748 - val_loss: 482.0646 - val_mae: 11.5048 - lr: 1.0000e-04
Epoch 151/200
42/42 [==============================] - 255s 6s/step - loss: 536.9548 - mae: 12.5808 - val_loss: 458.6201 - val_mae: 11.3058 - lr: 1.0000e-04
Epoch 152/200
42/42 [==============================] - 264s 6s/step - loss: 513.6041 - mae: 12.3022 - val_loss: 435.3277 - val_mae: 10.8769 - lr: 1.0000e-04
Epoch 153/200
42/42 [==============================] - 245s 6s/step - loss: 493.1646 - mae: 12.0133 - val_loss: 415.6396 - val_mae: 10.6286 - lr: 1.0000e-04
Epoch 154/200
42/42 [==============================] - 258

In [2]:
def preprocess_data(sequence, n_steps, channels):
    """
    Function to preprocess the data and prepare it for model training.
    
    Parameters:
        sequence (xarray.Dataset): Data sequence to be preprocessed.
        n_steps (int): Number of time steps in the input sequence.
        channels (int): Number of channels in the input sequence.
    
    Returns:
        tuple: A tuple containing the preprocessed input sequence (X) and output sequence (y).
    """
    # Select data for Jun, Jul, Aug, Sept
    sequence = sequence.sel(time=slice("2017-01-01","2018-12-31"))
    sequence = sequence.where(sequence.time.dt.month.isin([6,7,8,9]), drop=True)
    max = sequence.max()
    min = sequence.min()
    
    # Max-min normalization
    sequence = (sequence-min)/(max - min)
    sequence = np.array(sequence)
    
    # Exponential Space Transform
    valid_pts = np.where(sequence==sequence)
    sequence[valid_pts] = np.exp(sequence[valid_pts])
    sequence[valid_pts] = np.power(sequence[valid_pts],7)
    invalid_pts = np.where(sequence != sequence)
    sequence[invalid_pts] = 0
    
    # Prepare input and output sequences
    X, y = [], []
    for i in range(len(sequence)):
        end_ix = i + n_steps
        if end_ix + lead_time > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix + lead_time]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)


def create_input(channels, n_steps):
    """
    Function to create the input for the model.
    
    Parameters:
        channels (list): List of channels to be used in the input.
        n_steps (int): Number of time steps in the input sequence.
        
    Returns:
        numpy.ndarray: The input for the model.
    """
    stack = []
    for i in range(len(channels)):
        channel_data = xr.open_dataarray(f"/home/cccr/roxy/matin/MTech_project/data/{channels[i]}")
        input, _ = preprocess_data(channel_data, n_steps=n_steps, channels=channels[i])
        input = np.expand_dims(input, axis=2)
        print(f"Adding channel {i} with shape: {input.shape}")
        stack.append(input)
    out = np.dstack(stack)
    return out


channels = ["FilteredrfBOB_13lag.nc","FilteredSSTBOB_13.nc"]
steps = 7

# Create input for the model
input = create_input(channels, steps)

# Preprocess target data
channel_data = xr.open_dataarray(add_data + "FilteredrfBOB_13lag.nc")
_, target = preprocess_data(channel_data, n_steps=steps, channels=channels[-1])
target = np.expand_dims(target, axis=1)

# Roll axes for input and target
x = np.rollaxis(input, 4, 2)
x = np.rollaxis(x, 4, 2)

y = np.rollaxis(target, 3, 1)
y = np.rollaxis(y, 3, 1)

# Assert that input and target shapes are consistent
assert input.shape[0] == target.shape[0]
assert input.shape[-1] == target.shape[-1]
assert input.shape[-2] == target.shape[-2]


# Print shape of input and target
print("INPUT SHAPE --> ", input.shape)
print("TARGET SHAPE --> ", target.shape)

# Delete original input and target variables to free up memory
del input
del target


Adding channel 0 with shape: (234, 7, 1, 25, 39)
Adding channel 1 with shape: (234, 7, 1, 25, 39)
INPUT SHAPE -->  (234, 7, 2, 25, 39)
TARGET SHAPE -->  (234, 1, 25, 39)


In [3]:
model = tf.keras.models.load_model(model_path)
pred = model.predict(x)

actual = y.flatten()
prediction = pred.flatten()
corr = np.corrcoef(actual,prediction)
corr

8/8 [==============================] - 3s 218ms/step


array([[1.        , 0.81277434],
       [0.81277434, 1.        ]])

In [ ]:
[0.94, 0.84, 0.82, 0.81]